Simple example of eXirt's use

Run all bullets in the sequence below!



Generate files?


## Install dependences

In [2]:
!pip install openml
!pip install catsim

import openml
import pandas as pd
import statistics
import numpy as np
import random
import copy


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from google.colab import files



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py

--2023-01-24 16:40:08--  https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7453 (7.3K) [text/plain]
Saving to: ‘decodIRT_MLtIRT.py’

decodIRT_MLtIRT.py  100%[===================>]   7.28K  --.-KB/s    in 0.001s  

2023-01-24 16:40:09 (5.44 MB/s) - ‘decodIRT_MLtIRT.py’ saved [7453/7453]

--2023-01-24 16:40:09--  https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awa

## Install eXirt 

In [4]:
!pip install eXirt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for eXirt: filename=eXirt-1.0.0.7-py3-none-any.whl size=8966 sha256=d5e0ded44916d7b7bacb570728bda5bad3f4ed532481378cac0ed9da923bf482
  Stored in directory: /root/.cache/pip/wheels/c8/8d/05/d15c94e2c896ba2632e8872c08be38c01828c4dde901bb1512
Successfully built eXirt


In [5]:
from pyexirt import eXirt as explainer

## Load data and Pre-process

In [6]:
def normalize(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        if(len(df_norm[column].unique()) > 1): #fix NaN generation
          df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        else:
          df_norm[column] = 0
    return df_norm

In [7]:
#Select dataset name by openml link https://www.openml.org/search?sort=date
dataset_name = "pc1"

In [8]:
#load dataset by OpenML

dataset = openml.datasets.get_dataset(dataset_name)
X, Y, categorical_indicator, attribute_names = dataset.get_data(
                  dataset_format="dataframe", target=dataset.default_target_attribute)

print(dataset)
#Preprocess Y and X numerics

if (Y.dtype != 'numeric'):
  Y = Y.astype(int)

for i,c in enumerate(X.columns):
  if (X[c].dtype != 'float64'):
    X = X.astype(float)

#Normalization
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y) # 70% training and 30% test



OpenML Dataset
Name..........: pc1
Version.......: 1
Format........: ARFF
Upload Date...: 2014-10-06 23:57:45
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/53951/pc1.arff
OpenML URL....: https://www.openml.org/d/1068
# of features.: 22
# of instances: 1109


## Creation and prediction model

In [9]:
model = RandomForestClassifier(200)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

## Global explanation rank

In [10]:
explainer = explainer.eXirt()

In [11]:
global_explanation_attributes, global_explanation_attributes_scores = explainer.explainRankByEXirt(model, X_train, X_test, y_train, y_test,dataset_name)

In [12]:
global_explanation_attributes

['locCodeAndComment',
 'lOComment',
 'I',
 'loc',
 'lOBlank',
 'uniq_Opnd',
 'V',
 'total_Op',
 'lOCode',
 'E',
 'B',
 'iv(G)',
 'v(g)',
 'uniq_Op',
 'branchCount',
 'N',
 'L',
 'D',
 'T',
 'total_Opnd',
 'ev(g)']

In [13]:
global_explanation_attributes_scores

,Final Score
locCodeAndComment,-114.202104
lOComment,-113.97167
I,-113.928668
loc,-113.908383
lOBlank,-113.885679
uniq_Opnd,-113.817274
V,-113.746498
total_Op,-113.743966
lOCode,-113.726313
E,-113.723744
